# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
gk = tuple((g := globals()).keys())

In [ ]:
#| export
import re
import sys
import inspect
from typing import Any
import fastcore.all as FC
from fastcore.xtras import is_listy
import dialoghelper
from dialoghelper import *
from dialoghelper.core import _find_frame_dict

In [ ]:
from pathlib import Path
from IPython.display import Markdown
from fastcore.test import *
from fastcore.xtras import dict2obj
from fastcore.tools import *
from dialoghelper.stdtools import *
import dialoghelper.tmux
import dialoghelper.core

In [ ]:
from dialoghelper.inspecttools import *

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# from uro.super_w import super_w

# globals - what's available on startup?

Run below first after a restart.

In [ ]:
print(gk)

('__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', 'read_url', 'transient', 'run_cmd', 'sys', '__os', '__dialog_name', '__msg_id', 'g')


In [ ]:
if 'find_msg_id' in gk: assert __msg_id == find_msg_id()
if 'find_dname' in gk: assert __dialog_name == find_dname()
print(f"{g['__name__']=}\n{g['__doc__']=}\n{g['__package__']=}\n{g['__loader__']=}\n{g['__spec__']=}\n{g['transient']=}\n{g['__os']=}\n{g['__dialog_name']=}\n{g['__msg_id']=}")

g['__name__']='__main__'
g['__doc__']='Automatically created module for IPython interactive environment'
g['__package__']=None
g['__loader__']=None
g['__spec__']=None
g['transient']=<function transient at 0x7901901f3420>
g['__os']=<module 'os' (frozen)>
g['__dialog_name']='vic/dlgutil/nbs/00_core'
g['__msg_id']='_07c521e8'


In [ ]:
import ipykernel_helper

In [ ]:
dir(ipykernel_helper)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'core',
 'fix_editable_priority',
 'get_md',
 'gh_blob_to_raw',
 'load_ipython_extension',
 'read_gh_repo',
 'read_url',
 'run_cmd',
 'scrape_url',
 'transient']

In [ ]:
ipykernel_helper.core??

<unknown>:215: SyntaxWarning: invalid escape sequence '\('
<unknown>:215: SyntaxWarning: invalid escape sequence '\)'
<unknown>:215: SyntaxWarning: invalid escape sequence '\('
<unknown>:215: SyntaxWarning: invalid escape sequence '\)'



```python
"""API for ipykernel-helper"""

# AUTOGENERATED! DO NOT EDIT! File to edit: ../nbs/00_core.ipynb.

# %% auto 0
__all__ = ['transient', 'run_cmd', 'get_md', 'scrape_url', 'gh_blob_to_raw', 'read_gh_repo', 'read_url', 'fix_editable_priority',
           'load_ipython_extension']

# %% ../nbs/00_core.ipynb
from fastcore.meta import delegates
from fastcore.utils import patch,dict2obj
from types import ModuleType, FunctionType, MethodType, BuiltinFunctionType
from inspect import signature, currentframe
from functools import cmp_to_key,partial
from collections.abc import Mapping
from textwrap import dedent
from cloudscraper import create_scraper
from toolslm.funccall import *
from toolslm.xml import *
from ast import literal_eval
from urllib.parse import urlparse, urljoin
from fastcore.net import HTTP404NotFoundError
from ghapi.all import GhApi

import typing,warnings,re,os,html2text,base64

from IPython.core.interactiveshell import InteractiveShell
from IPython.core.completer import ProvisionalCompleterWarning
from jedi import Interpreter, Script as jscript

from IPython.core.display import DisplayObject
from IPython.display import display,Markdown,HTML
from IPython.core.oinspect import Inspector

# %% ../nbs/00_core.ipynb
warnings.filterwarnings('ignore', category=ProvisionalCompleterWarning)

# %% ../nbs/00_core.ipynb
def _safe_repr(obj, max_len=200):
    "Safely get the repr() of an object, truncating if it exceeds max_len."
    try:
        s = str(obj)
        return s[:max_len] + ("…" if len(s)>max_len else "")
    except Exception as e: return f"<repr error: {str(e)}>"

# %% ../nbs/00_core.ipynb
@patch
def user_items(self:InteractiveShell, max_len=200, xtra_skip=()):
    "Get user-defined vars & funcs from namespace."
    ns,nsh = self.user_ns,self.user_ns_hidden
    ignore = {'nbmeta', 'receive_nbmeta'}
    ignore.add(xtra_skip)
    rm_types = (
        type, FunctionType, ModuleType, MethodType, BuiltinFunctionType,
        getattr(typing, '_SpecialGenericAlias', ()),
        getattr(typing, '_GenericAlias', ()),
        getattr(typing, '_SpecialForm', ())
    )
    user_items = {k:v for k, v in ns.items()
                  if not k in ignore and k not in nsh}
    user_vars = {k:_safe_repr(v, max_len=max_len)
                 for k, v in user_items.items() if not k.startswith('_') and not isinstance(v, rm_types)}
    user_fns = {k:str(signature(v)) for k, v in user_items.items()
                if isinstance(v, FunctionType) and v.__module__ == '__main__' and not k.startswith('__')}
    return user_vars,user_fns

# %% ../nbs/00_core.ipynb
def _rank(c, s):
    "Rank a completion `c` for text `s` with namespace `ns`."
    parts = s.split('.')
    is_public = not c.text.startswith('_')
    if c.type=='param': r=1
    elif c.mod=='__main__': r=2 # local
    elif len(parts)>1 and parts[0]==c.mod: r=3 # module
    elif c.mod=='builtins': r=4
    else: r=5
    return r if is_public else r+0.1

# %% ../nbs/00_core.ipynb
@patch
def ranked_complete(self:InteractiveShell, code, line_no=None, col_no=None):
    ns = self.user_ns
    lines = code.splitlines(True)
    if line_no: offset = sum(len(lines[i]) for i in range(line_no-1)) + col_no -1
    else: offset = len(code)
    cs = self.Completer.completions(code, offset)
    def _c(a):
        res = dict2obj({attr: getattr(a, attr) for attr in dir(a) if attr[0]!='_'})
        res['mod']= getattr(ns.get(a.text, None), '__module__', None)
        res['rank'] = _rank(res, s=code)
        return res
    # Remove dunder vars, unless the user seems to be looking for them explicitly
    return [_c(c) for c in cs if not c.text.startswith('__') or '__' in code]

# %% ../nbs/00_core.ipynb
def _signatures(ns, s, line, col):
    ctx = Interpreter(s, [ns]).get_signatures(line, col)
    if not ctx: ctx = jscript(s).get_signatures(line, col)
    return ctx

@patch
def sig_help(self:InteractiveShell, code, line_no=None, col_no=None):
    ns = self.user_ns
    ctx = _signatures(ns, code, line=line_no, col=col_no)
    def _s(s): return {'label':s.description,'typ':s.type, 'mod':s.module_name, 'doc':s.docstring(),
                       'idx':s.index, 'params':[{'name':p.name, 'desc':p.description} for p in s.params]}
    return [_s(opt) for opt in ctx]

# %% ../nbs/00_core.ipynb
@patch
def get_vars(self:InteractiveShell, vs:list, literal=True):
    "Get variables from namespace."
    ns = self.user_ns
    def _maybe_eval(o):
        try: literal_eval(repr(o)); return o
        except: return str(o)
    return {v:_maybe_eval(ns[v]) if literal else str(ns[v]) for v in vs if v in ns}

# %% ../nbs/00_core.ipynb
def _get_schema(ns: dict, t):
    "Check if tool `t` has errors."
    if t not in ns: return f"`{t}` not found. Did you run it?"
    try: return {'type':'function', 'function':get_schema(ns[t], pname='parameters', evalable=True, skip_hidden=True)}
    except Exception as e: return f"`{t}`: {e}."

@patch
def get_schemas(self:InteractiveShell, fs:list):
    "Get schemas from namespace."
    ns = self.user_ns
    return {f:_get_schema(ns,f) for f in fs}

# %% ../nbs/00_core.ipynb
@patch
def xpush(self:InteractiveShell, interactive=False, **kw):
    "Like `push`, but with kwargs"
    self.push(kw, interactive=interactive)

# %% ../nbs/00_core.ipynb
@patch
def publish(self:InteractiveShell, data='', subtype='plain', mimetype='text', meta=None, update=False, **kw):
    if isinstance(data, DisplayObject): data,_ = self.display_formatter.format(data)
    elif not isinstance(data, Mapping): data = {f'{mimetype}/{subtype}': data}
    self.display_pub.publish(data, metadata=meta, transient=kw, update=update)

# %% ../nbs/00_core.ipynb
def transient(data='', subtype='plain', mimetype='text', meta=None, update=False, **kw):
    display({f'{mimetype}/{subtype}': data}, raw=True, metadata=meta, transient=kw, update=update)

# %% ../nbs/00_core.ipynb
def run_cmd(cmd, data='', meta=None, update=False, **kw):
    transient(data, meta=meta, update=update, cmd=cmd, **kw)

# %% ../nbs/00_core.ipynb
def _absolutify_imgs(md, base_url):
    def fix(m):
        alt,img_url = m.group(1),m.group(2)
        if not img_url.startswith('http'): img_url = urljoin(base_url, img_url)
        alt = alt.replace('\\','')
        return f'![{alt}]({img_url})'
    return re.sub(r'!\[(.*?)\]\((.*?)\)', fix, md)

# %% ../nbs/00_core.ipynb
def get_md(html, url='', mmode=None, ignore_links=False, ignore_images=False, mark_code=True):
    "Convert HTML to markdown with absolute image URLs and optional math mode"
    h = html2text.HTML2Text()
    h.body_width = 0
    h.ignore_links, h.ignore_images, h.mark_code = ignore_links, ignore_images, mark_code
    res = _absolutify_imgs(h.handle(str(html)), url)
    if mmode == 'safe': res = res.replace(r'\\(',r'\(').replace(r'\\)',r'\)')
    return re.sub(r'\[code]\s*\n(.*?)\n\[/code]', lambda m: f'```\n{dedent(m.group(1))}\n```', res, flags=re.DOTALL).strip()

# %% ../nbs/00_core.ipynb
def scrape_url(url): return create_scraper().get(url)

# %% ../nbs/00_core.ipynb
def _get_math_mode():
    v = os.getenv('USE_KATEX', '')
    if v.lower() in {'0', 'false', 'none', ''}: return None
    return 'dollar' if v.lower().startswith('d') else 'safe'

# %% ../nbs/00_core.ipynb
def _aify_imgs(md): return re.sub(r'!\[(.*?)\]\((.*?)\)', r'![\1](\2#ai)', md)

# %% ../nbs/00_core.ipynb
def gh_blob_to_raw(url):
    "Convert github.com/user/repo/blob/... URL to raw.githubusercontent.com URL"
    m = re.match(r'https?://(?:www\.)?github\.com/([^/]+)/([^/]+)/blob/([^/]+)/(.+)', url)
    if not m: return url
    owner, repo, ref, path = m.groups()
    return f'https://raw.githubusercontent.com/{owner}/{repo}/{ref}/{path}'

# %% ../nbs/00_core.ipynb
def _extract_section(soup, url, selector=None):
    "Extract a specific section from soup, or the whole thing"
    if selector: return '\n\n'.join(str(s) for s in soup.select(selector))
    parsed = urlparse(url)
    if not parsed.fragment: return str(soup)
    section = soup.find(id=parsed.fragment)
    if not section: return ''
    elements = [section]
    current = section.next_sibling
    while current:
        if hasattr(current, 'name') and current.name == section.name: break
        elements.append(current)
        current = current.next_sibling
    return ''.join(str(el) for el in elements)

# %% ../nbs/00_core.ipynb
def _convert_math(soup, mode):
    for math in soup.find_all('math'):
        annot = math.find('annotation', {'encoding': 'application/x-tex'})
        if not annot: continue
        tex,display = annot.text.strip(), math.get('display') == 'block'
        if mode == 'dollar': wrap = f'$${tex}$$' if display else f'${tex}$'
        else: wrap = f'$${tex}$$' if display else f'\({tex}\)'
        math.replace_with(wrap)

# %% ../nbs/00_core.ipynb
def read_gh_repo(owner, repo, ref=None, path=''):
    "Read GitHub repo info: description, file list, and README"
    api = GhApi()
    info = api.repos.get(owner, repo)
    res = [f"# {info.full_name}", info.description or '']
    ref = ref or info.default_branch
    contents = api.repos.get_content(owner, repo, path or '', ref=ref)
    files = [f"- {'📁 ' if c.type=='dir' else ''}{c.name}" for c in contents]
    res.append(f'\n## /{path or ""} Files\n' + '\n'.join(files))
    if not path:
        try:
            readme = api.repos.get_readme(owner, repo, ref=ref)
            res.append('\n## README\n' + base64.b64decode(readme.content).decode())
        except HTTP404NotFoundError: pass
    return '\n'.join(res)

# %% ../nbs/00_core.ipynb
def read_url(url:str, as_md:bool=True, extract_section:bool=True, selector:str=None, ai_img:bool=False):
    "Read url from web"
    from bs4 import BeautifulSoup
    gh = parse_gh_url(url)
    if gh:
        if gh['typ']=='blob': url = gh_blob_to_raw(url)
        elif gh['typ'] in (None, 'tree'): return read_gh_repo(gh['owner'], gh['repo'], gh['ref'], gh['path'])
    o = scrape_url(url)
    ctype = (o.headers.get('content-type') or 'text/plain').split(';')[0]
    res = o.text
    if ctype == 'text/html':
        soup = BeautifulSoup(res, 'lxml')
        if ('#' in url and extract_section) or selector: soup = BeautifulSoup(_extract_section(soup, url, selector), 'lxml')
        mmode = _get_math_mode()
        if mmode: _convert_math(soup, mmode)
        base = soup.find('base')
        base_url = urljoin(url, base['href'] if base else '')
        res = get_md(soup, base_url, mmode) if as_md else str(soup)
    if ai_img: res = _aify_imgs(res)
    return res

# %% ../nbs/00_core.ipynb
def fix_editable_priority():
    import sys
    from importlib.machinery import PathFinder
    try: sys.meta_path.append(sys.meta_path.pop(sys.meta_path.index(PathFinder)))
    except ValueError: pass

# %% ../nbs/00_core.ipynb
@patch
def _get_info(self:Inspector, obj, oname='', formatter=None, info=None, detail_level=0, omit_sections=()):
    "Custom formatter for ?? output"
    orig = self._orig__get_info(obj, oname=oname, formatter=formatter, info=info,
                               detail_level=detail_level, omit_sections=omit_sections)
    if detail_level==0: return orig
    info_dict = self.info(obj, oname=oname, info=info, detail_level=detail_level)
    out = []
    if c:=info_dict.get('source'): out.append(f"\n```python\n{dedent(c)}\n```")
    if c:=info_dict.get('file'): out.append(f"**File:** `{c}`")
    return {'text/markdown': '\n\n'.join(out), 'text/html': '', 'text/plain': orig['text/plain']}

# %% ../nbs/00_core.ipynb
def load_ipython_extension(ip):
    from ipykernel_helper import transient,run_cmd
    ns = ip.user_ns
    ns['read_url'],ns['transient'],ns['run_cmd'] = read_url,transient,run_cmd
```

**File:** `/usr/local/lib/python3.12/site-packages/ipykernel_helper/core.py`

In [ ]:
sys.meta_path

 _frozen_importlib.BuiltinImporter,
 _frozen_importlib.FrozenImporter,
 _frozen_importlib_external.PathFinder,
 __editable___solveit_0_0_58_finder._EditableFinder,

In [ ]:
[k for k in sys.modules if 'solveit' in k]

['__editable___solveit_0_0_58_finder']

In [ ]:
slvt_k = [k for k in sys.modules if '__editable___solveit_' in k][0]
print(dir(slvt := sys.modules[slvt_k]))

['MAPPING', 'ModuleSpec', 'NAMESPACES', 'PATH_PLACEHOLDER', 'Path', 'PathFinder', '_EditableFinder', '_EditableNamespaceFinder', '__annotations__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'annotations', 'chain', 'install', 'module_suffixes', 'spec_from_file_location', 'sys']


In [ ]:
if (msg := read_msg(1, True)['msg'])['content'].startswith('!cat'):
    update_msg(msg['id'], content=f"!cat {slvt.__file__}")
    run_msg(msg['id'])

In [ ]:
!cat /usr/local/lib/python3.12/site-packages/__editable___solveit_0_0_58_finder.py

cat: /usr/local/lib/python3.12/site-packages/__editable___solveit_0_0_57_finder.py: No such file or directory


from __future__ import annotations
import sys
from importlib.machinery import ModuleSpec, PathFinder
from importlib.machinery import all_suffixes as module_suffixes
from importlib.util import spec_from_file_location
from itertools import chain
from pathlib import Path

MAPPING: dict[str, str] = {'solveit': '/root/solveit/solveit', 'tests': '/root/solveit/tests'}
NAMESPACES: dict[str, list[str]] = {}
PATH_PLACEHOLDER = '__editable__.solveit-0.0.58.finder' + ".__path_hook__"


class _EditableFinder:  # MetaPathFinder
    @classmethod
    def find_spec(cls, fullname: str, path=None, target=None) -> ModuleSpec | None:  # type: ignore
        # Top-level packages and modules (we know these exist in the FS)
        if fullname in MAPPING:
            pkg_path = MAPPING[fullname]
            return cls._find_spec(fullname, Path(pkg_path))

        # Handle immediate children modules (required for namespaces to work)
        # To avoid problems with case sensitivity in the file system we deleg

In [ ]:
' '.join(_.__module__ for _ in sys.meta_path)

'_distutils_hack _frozen_importlib _frozen_importlib _frozen_importlib_external __editable___solveit_0_0_58_finder six'

In [ ]:
#| export
def solveit_version():
    "Return the version of solveit if it is found"
    s = ' '.join(_.__module__ for _ in sys.meta_path)
    mtch = re.match(r'.*__editable___solveit_(\d+)_(\d+)_(\d+)_finder', s)
    return f"{mtch[1]}.{mtch[2]}.{mtch[3]}" if mtch else ''

In [ ]:
solveit_version()

'0.0.58'

In [ ]:
#| export
def in_dialog():
    "Check if the code is running in a solveit dialog"
    return bool(solveit_version() and dialoghelper.find_dname() and dialoghelper.find_msg_id())

In [ ]:
in_dialog()

True

In [ ]:
#| export
def get_caller_globals(): 
    "Return the globals of the caller"
    return inspect.currentframe().f_back.f_globals

In [ ]:
test_var = "I'm in globals"
def caller_func():
    local_var = "I'm local"
    return get_caller_globals()

test_is('test_var' in caller_func(), True)
test_is('local_var' in caller_func(), False)

In [ ]:
dialoghelper.core._find_frame_dict??


```python
def _find_frame_dict(var:str):
    "Find the dict (globals or locals) containing var"
    frame = currentframe().f_back.f_back
    while frame:
        if var in frame.f_globals: return frame.f_globals
        frame = frame.f_back
    raise ValueError(f"Could not find {var} in any scope")
```

**File:** `/usr/local/lib/python3.12/site-packages/dialoghelper/core.py`

In [ ]:
find_var??


```python
def find_var(var:str):
    "Search for var in all frames of the call stack"
    return _find_frame_dict(var)[var]
```

**File:** `/usr/local/lib/python3.12/site-packages/dialoghelper/core.py`

In [ ]:
print((msg := read_msg(-1, True)['msg'])['output'][0]['data']['text/plain'])

Signature: find_var(var: str)
Source:   
def find_var(var:str):
    "Search for var in all frames of the call stack"
    return _find_frame_dict(var)[var]
File:      /usr/local/lib/python3.12/site-packages/dialoghelper/core.py
Type:      function


In [ ]:
print(msg['output'][0]['data']['text/markdown'])


```python
def find_var(var:str):
    "Search for var in all frames of the call stack"
    return _find_frame_dict(var)[var]
```

**File:** `/usr/local/lib/python3.12/site-packages/dialoghelper/core.py`


# helpers

In [ ]:
#| export
_empty = inspect.Parameter.empty

In [ ]:
def val_at(o, sym: str, default=_empty, sep='.'):
    "Traverse nested `o` looking for attributes/items specified in dot-separated `sym`."
    if not isinstance(sym, str): raise TypeError(f'{sym=!r} is not a string')
    try:
        for a in sym.split(sep):
            if a[0]=='-' or a[0].isdigit(): a = int(a)
            try: o = o[a]
            except Exception:
                if isinstance(a, int):
                    a = str(a)
                    try: o = o[a]
                    except Exception: pass
                o = getattr(o, a)
    except Exception as e:
        if default is not empty: return default
        raise e
    return o

In [ ]:
records = [
    {'id': 1, 'items': [2, 3], 'meta': {'count': 4}}, 
    {'id': 5, 'items': [6, 7], 'nested': [{'val': 81}, {'val': 82}]}, 
    {'id': 9, 'items': [10, 11], 'meta': {'count': 12}}
]

test_eq(val_at(records, '0.id'), 1)
test_eq(val_at(records, '1.items'), [6, 7])
test_eq(val_at(records, '2.meta'), {'count': 12})

Works with lists of dicts (common in API responses)


In [ ]:
test_fail(lambda: val_at({}, 'a.b'))
test_fail(lambda: val_at([], 'a.b'))
test_fail(lambda: val_at({'a': 1}, 'a.b'))
test_fail(lambda: val_atpath({'a': 1}, 'a', 'b'))

# With default, no error
test_eq(val_at({'a': 1}, 'a.b', None), None)

Error handling: raises when path not found (unless default provided)

In [ ]:
j2 = {
    "app": {
        "Garden": {
            "Flowers": {
                "Red flower": "Rose",
                "White Flower": "Jasmine",
                "Yellow Flower": "Marigold"
            }
        },
        "Fruits": {
            "Yellow fruit": ["Mango", {"Banana": ["Canary Island", "Puerto Rico"]}],
            "Green fruit": "Guava",
            "White Flower": "groovy"
        },
        "Trees": {
            "label": {
                "Yellow fruit": "Pumpkin",
                "White Flower": "Bogan"
            }
        },
        "Numbers": [1, 2, 3, 4, 5],
        "Boolean": True,
        "Null": None
    }
}

j2_str = j2#json.dumps(j2)

test_eq(val_at(j2_str, 'app.Fruits.Yellow fruit.1.Banana.0'), 'Canary Island')
test_eq(val_at(j2_str, 'app.Garden.Flowers.Red flower'), 'Rose')
test_eq(val_at(j2_str, 'app.Numbers.2'), 3)
test_eq(val_at(j2_str, 'app.Boolean'), True)
test_eq(val_at(j2_str, 'app.Null'), None)
test_fail(lambda: val_at(j2_str, 'app.NonExistent'))
test_fail(lambda: val_at(j2_str, 'app.Fruits.Yellow fruit.3'))
test_is(val_at(j2_str, 'app.Fruits.Yellow fruit.3', None), None)

In [ ]:
j2_obj = dict2obj(j2)

test_eq(val_at(j2_obj, 'app.Fruits.Yellow fruit.1.Banana.0'), 'Canary Island')

Write a new version of `val_at` that also acepts indexing, i.e. sym can be dot separated with indexing:
- app.Fruits.Yellow fruit[1].Banana[0] <=> app.Fruits.Yellow fruit.1.Banana.0
- app[Fruits][Yellow fruit][1][Banana][0] <=> app.Fruits.Yellow fruit.1.Banana.0
- app[Fruits].Yellow fruit.1.Banana[0] <=> app.Fruits.Yellow fruit.1.Banana.0
- 2.meta <=> [2].meta
- meta.2 <=> meta[2]

In [ ]:
list(filter(None, 'a..b'.split('.')))

['a', 'b']

In [ ]:
#| export
def at_(
    o, # Object to traverse (dict, list, object, or nested combination)
    sym: str, # Path using dots and/or brackets (e.g., 'a.b[0].c' or 'a[b][c]')",
    default: Any=_empty, # Value to return if path not found (raises exception if not provided)
    sep='.' # Separator for path segments
) -> Any: # Value at the specified path
    "Traverse nested `o` using path `sym` with dot notation and/or bracket indexing"
    sym = re.sub(r'\[([^\]]+)\]', r'.\1', sym)
    try:
        for a in filter(None, sym.split(sep)):
            if a.lstrip('-').isdigit(): a = int(a)
            try: o = o[a]
            except Exception:
                if isinstance(a, int):
                    try: o = o[str(a)]; continue
                    except Exception: pass
                o = getattr(o, a)
    except Exception:
        if default is not _empty: return default
        raise
    return o

`at_` provides flexible path-based access to nested data structures:

**Supported types:** `o` can be/contains any combination of `Sequence`, `Mapping` (dicts, lists, tuples, L, etc), objects with \_\_getitem__, and/or dataclasses, objects with attributes accesible by `getattr`.

**Path syntax:**
- Dot notation: `'a.b.c'` accesses `o['a']['b']['c']` or `o.a.b.c`
- Bracket notation: `'a[b][c]'` is equivalent to `'a.b.c'`: `[x]` is a shorthand for `.x`
- Mixed: `'a.b[0].c'` combines both styles
- Numeric indices: `'items.2'` or `'items[2]'` for list/array access
- Empty path: `''` returns the object itself

**Access priority:** Item access (`[]`) is tried before attribute access (`.`)

**Error handling:** Raises exception if path not found, unless `default` is provided

In [ ]:
test_eq(at_({'a': 13}, 'a'), 13)

test_eq(at_({'a': {'b': 13}}, 'a.b'), 13)
test_eq(at_({'a': dict2obj({'b': 13})}, 'a.b'), 13)

test_eq(at_({'a': {'3': 7}}, 'a.3'), 7)

test_fail(lambda: at_(o, 'app.NonExistent'))
test_fail(lambda: at_(o, '[0'))
test_fail(lambda: at_(None, 'a'))
test_eq(at_(None, 'a', None), None)
test_fail(lambda: at_(None, 'a'))

test_eq(at_(o := {'meta': [1,2,3]}, ''), o)
test_eq(at_(o, 'meta[2]'), 3)
test_eq(at_(o, 'meta.2'), 3)

test_eq(at_('xyz', 'a', None), None)
test_eq(at_((s := 'xyz'), 'split'), s.split)

test_eq(at_([{'a':1}], '[0][a]'), 1)

test_eq(at_(records, '[2].meta'), {'count': 12})
test_eq(at_(records, '2[meta]'), {'count': 12})

j2_list = [{'a':1}, {'b':2}]
test_eq(at_(j2_list, '0'), {'a':1})
test_eq(at_(j2_list, '1'), {'b':2})
test_eq(at_(j2_list, '[0][a]'), 1)
test_eq(at_(j2_list, '0.a'), 1)
test_eq(at_(j2_list, '[1].b'), 2)

test_eq(at_(j2, 'app[Numbers][2]'), 3)
test_eq(at_(j2, 'app[Fruits][Yellow fruit][1][Banana][0]'), 'Canary Island')
test_eq(at_(j2, 'app.Fruits[Yellow fruit].1.Banana[0]'), 'Canary Island')

test_eq(at_(dict2obj(j2), 'app[Fruits][Yellow fruit][1][Banana][0]'), 'Canary Island')
test_eq(at_(dict2obj(j2), 'app.Fruits[Yellow fruit].1.Banana[0]'), 'Canary Island')

test_is(at_(j2, '[bad][path]', 'default'), 'default')
test_is(at_(j2, '[missing]', None), None)
test_is(at_(j2, 'app[missing]', 'default'), 'default')
test_is(at_(j2, 'app[NonExistent]', None), None)

In [ ]:
print(13)

13


In [ ]:
test_eq(at_(read_msg(), 'msg.output'), [{'output_type': 'stream', 'name': 'stdout', 'text': '13\n'}])

I'm a note.

In [ ]:
test_eq(at_(read_msg(), 'msg.output'), '')

In [ ]:
if not at_(read_msg(1), 'msg.content').startswith("I'm a prompt"):
    msgid = add_msg("I'm a prompt", msg_type='prompt', output='are you? really?')
    # _=add_scr(f"selectMsg($('#{msgid}'), {centered: true})")
display(Markdown('### Test markdown'))
print(_msgid := find_msg_id())

### Test markdown

_86e4f755


I'm a prompt

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

are you? really?

In [ ]:
test_eq(at_(read_msg(), 'msg.output'), 'are you? really?')

In [ ]:
oo = at_(read_msg(0, True, _msgid), 'msg.output')
oo

[{'output_type': 'display_data', 'metadata': {}, 'data': {'text/plain': '<IPython.core.display.Markdown object>', 'text/markdown': '### Test markdown'}}, {'output_type': 'stream', 'name': 'stdout', 'text': '_86e4f755\n'}]

Future improvements, more complex paths:

1. **Negative indexing support** - currently `-1` works but only because Python handles it; we could make it explicit
2. **Wildcard/glob patterns** - e.g. `items.*.name` to get all names from a list of dicts
3. **Optional chaining** - e.g. `a?.b?.c` returns `None` instead of raising if any part is missing
4. **Slice support** - e.g. `items[1:3]` or `items[:2]`
5. **Multiple paths** - return values from multiple paths at once
6. **Callable filter** - e.g. `items[lambda x: x['active']].name`

Can we improve `_at` somehow? Give me alternatives. I'm interested in conciseness, strength, flexibility; more complex paths will be handle by another function.

**1. Reduce-based (most concise)**

In [ ]:
def at_(o, sym, default=_empty, sep='.'):
    "Traverse nested `o` using path `sym` with dot notation and/or bracket indexing"
    def step(o, a):
        if a.lstrip('-').isdigit(): a = int(a)
        try: return o[a]
        except: return getattr(o, str(a))
    try: return reduce(step, filter(None, re.sub(r'\[([^\]]+)\]', r'.\1', sym).split(sep)), o)
    except: 
        if default is not _empty: return default
        raise

**2. Operator.getitem + getattr combo (cleaner step logic)**

In [ ]:
def at_(o, sym, default=_empty, sep='.'):
    "Traverse nested `o` using path `sym` with dot notation and/or bracket indexing"
    def step(o, a):
        if a.lstrip('-').isdigit(): a = int(a)
        for fn in (lambda: o[a], lambda: o[str(a)] if isinstance(a,int) else None, lambda: getattr(o,a)):
            try: return fn()
            except: pass
        raise KeyError(a)
    try: return reduce(step, filter(None, re.sub(r'\[([^\]]+)\]', r'.\1', sym).split(sep)), o)
    except:
        if default is not _empty: return default
        raise

# tools

In [ ]:
dialoghelper.__version__

'0.0.57'

In [ ]:
# fc_tool_info()

Tools available from fastcore.tools: &`[rg, sed, view, create, insert, str_replace, strs_replace, replace_lines]`

In [ ]:
# tool_info()

Tools available from dialoghelper: &`[curr_dialog, msg_idx, add_html, find_msg_id, find_msgs, read_msg, del_msg, add_msg, update_msg, msg_insert_line, msg_str_replace, msg_strs_replace, msg_replace_lines]`

In [ ]:
# inspect_tool_info()

Tools available from inspecttools: &`[symsrc,showsrc,gettype,getdir,doimport,getval,getnth,run_code_interactive]`

In [ ]:
# dialoghelper.tmux.tmux_tool_info()

dialoghelper.tmux tools: &`[pane, list_panes, panes, list_windows, windows, list_sessions, sessions]`

In [ ]:
def get_tool_names(ns): return [k for k,v in ns.items() if callable(v)]

In [ ]:
print(list(get_tool_names(globals())))

['get_ipython', 'exit', 'quit', 'open', 'read_url', 'transient', 'run_cmd', 'is_listy', '_find_frame_dict', 'Path', 'Markdown', 'test_fail', 'test', 'nequals', 'test_eq', 'test_eq_type', 'test_ne', 'is_close', 'test_close', 'test_is', 'test_shuffled', 'test_stdout', 'test_warns', 'test_fig_exists', 'ExceptionExpected', 'dict2obj', 'rg', 'sed', 'view', 'create', 'insert', 'str_replace', 'strs_replace', 'replace_lines', 'Placements', 'empty', 'add_styles', 'find_var', 'set_var', 'find_dname', 'find_msg_id', 'call_endp', 'curr_dialog', 'msg_idx', 'add_scr', 'iife', 'pop_data', 'fire_event', 'event_get', 'find_msgs', 'add_html', 'read_msg', 'read_msgid', 'add_msg', 'del_msg', 'update_msg', 'run_msg', 'url2note', 'ast_py', 'ast_grep', 'get_folder', 'get_repo', 'msg_insert_line', 'msg_str_replace', 'msg_strs_replace', 'msg_replace_lines', 'msg_del_lines', 'load_gist', 'gist_file', 'import_string', 'is_usable_tool', 'mk_toollist', 'import_gist', 'tool_info', 'fc_tool_info', 'is_tool', 'doimpo

In [ ]:
#| export
def get_tool_names(ns=None, exclude=None, only_exported=False, exclude_private=True):
    "Return the names of all tools in namespace `ns` or current dialog."
    ns = ns or _find_frame_dict('__msg_id')
    exports = set(getattr(ns, '__all__', []))
    if exclude: exclude = set(get_tool_names(exclude) if not is_listy(exclude) else exclude)
    if inspect.ismodule(ns): ns = vars(ns)
    for k,v in ns.items():
        if exclude_private and k[0] == '_': continue
        if only_exported and k not in exports: continue
        if exclude and k in exclude: continue
        if not hasattr(__builtins__, k) and callable(v):
            try: 
                if is_usable_tool(v): yield k
            except Exception: pass

In [ ]:
print(list(get_tool_names()))

['read_url', 'run_cmd', 'rg', 'sed', 'view', 'create', 'insert', 'str_replace', 'strs_replace', 'replace_lines', 'empty', 'add_styles', 'find_var', 'find_dname', 'find_msg_id', 'curr_dialog', 'iife', 'find_msgs', 'add_html', 'read_msg', 'read_msgid', 'add_msg', 'del_msg', 'update_msg', 'run_msg', 'url2note', 'ast_py', 'msg_insert_line', 'msg_str_replace', 'msg_strs_replace', 'msg_replace_lines', 'msg_del_lines', 'load_gist', 'gist_file', 'is_usable_tool', 'import_gist', 'doimport', 'resolve', 'symsrc', 'showsrc', 'gettype', 'getdir', 'getval', 'getnth', 'run_code_interactive', 'solveit_version', 'in_dialog', 'get_caller_globals']


In [ ]:
print(list(get_tool_names(dialoghelper.stdtools)))


['empty', 'add_styles', 'find_var', 'find_dname', 'find_msg_id', 'curr_dialog', 'iife', 'find_msgs', 'add_html', 'read_msg', 'read_msgid', 'add_msg', 'del_msg', 'update_msg', 'run_msg', 'url2note', 'ast_py', 'msg_insert_line', 'msg_str_replace', 'msg_strs_replace', 'msg_replace_lines', 'msg_del_lines', 'load_gist', 'gist_file', 'is_usable_tool', 'import_gist', 'doimport', 'resolve', 'symsrc', 'showsrc', 'gettype', 'getdir', 'getval', 'getnth', 'run_code_interactive', 'run_cmd', 'rg', 'sed', 'view', 'create', 'insert', 'str_replace', 'strs_replace', 'replace_lines']


In [ ]:
print(list(get_tool_names(globals(), exclude=dialoghelper.stdtools)))

['read_url', 'solveit_version', 'in_dialog', 'get_caller_globals']


In [ ]:
print(list(get_tool_names(dialoghelper.tmux)))

['add_msg', 'WrapperDescriptorType', 'MethodWrapperType', 'MethodDescriptorType', 'BuiltinFunctionType', 'BuiltinMethodType', 'ipython_shell', 'in_ipython', 'in_colab', 'in_jupyter', 'in_notebook', 'BasicRepr', 'NullType', 'ignore_exceptions', 'Inf', 'GetAttrBase', 'GetAttr', 'ShowPrint', 'num_cpus', 'str2float', 'str2list', 'str2date', 'walk', 'globtastic', 'pglob', 'loads_multi', 'parse_env', 'IterLen', 'modify_exception', 'stringfmt_names', 'PartialFormatter', 'utc2local', 'local2utc', 'console_help', 'type2str', 'is_typeddict', 'reawaitable', 'empty', 'python', 'set_default_history', 'tmux_tool_info']


In [ ]:
#| export
def add_tools_card(ns=None):
    "Add a message with all tools in namespace `ns` or caller globals"
    ns = ns or _find_frame_dict('__msg_id')
    add_msg(mk_toollist(ns[_] for _ in get_tool_names(ns)))

In [ ]:
# add_tools_card()

- &`read_url`: Read url from web
- &`run_cmd`: Run `cmd` passing split `argstr`, optionally checking for allowed argstr
- &`rg`: Run the `rg` command with the args in `argstr` (no need to backslash escape)
- &`sed`: Run the `sed` command with the args in `argstr` (e.g for reading a section of a file)
- &`view`: View directory or file contents with optional line range and numbers
- &`create`: Creates a new file with the given content at the specified path
- &`insert`: Insert new_str at specified line number
- &`str_replace`: Replace first occurrence of old_str with new_str in file
- &`strs_replace`: Replace for each str pair in old_strs,new_strs
- &`replace_lines`: Replace lines in file using start and end line-numbers (index starting at 1)
- &`_empty`: Marker object for Signature.empty and Parameter.empty.
- &`add_styles`: Add solveit styles to `s`
- &`find_var`: Search for var in all frames of the call stack
- &`find_dname`: Get the message id by searching the call stack for __dialog_id.
- &`find_msg_id`: Get the message id by searching the call stack for __msg_id.
- &`curr_dialog`: Get the current dialog info.
- &`iife`: Wrap javascript code string in an IIFE and execute it via `add_html`
- &`find_msgs`: Find `list[dict]` of messages in current specific dialog that contain the given information. Call with no args to see the full dialog.
    Message ids are identical to those in LLM chat history, so do NOT call this to view a specific message -- instead use `read_msgid`.
    Note that LLM chat history only includes messages above the current prompt, whereas `find_msgs` can access *all* messages.
    To refer to a message found later, use its `id` field.
- &`add_html`: Send HTML to the browser to be swapped into the DOM
- &`read_msg`: Get the message indexed in the current dialog.
    - To get the exact message use `n=0` and `relative=True` together with `msgid`.
    - To get a relative message use `n` (relative position index).
    - To get the nth message use `n` with `relative=False`, e.g `n=0` first message, `n=-1` last message.
- &`read_msgid`: Get message `msgid`. Message IDs can be view directly in LLM chat history/context, or found in `find_msgs` results.
- &`add_msg`: Add/update a message to the queue to show after code execution completes.
- &`del_msg`: Delete a message from the dialog.
- &`update_msg`: Update an existing message. Provide either `msg` OR field key/values to update.
    - Use `content` param to update contents.
    - Only include parameters to update--missing ones will be left unchanged.
- &`run_msg`: Adds a message to the run queue. Use read_msg to see the output once it runs.
- &`url2note`: Read URL as markdown, and add note(s) below current message with the result
- &`ast_py`: Get an SgRoot root node for python `code`
- &`msg_insert_line`: Insert text at a specific line number in a message. Be sure you've called `read_msg(..., nums=True)` to ensure you know the line numbers.
- &`msg_str_replace`: Replace first occurrence of old_str with new_str in a message
- &`msg_strs_replace`: Replace multiple strings simultaneously in a message
- &`msg_replace_lines`: Replace a range of lines with new content in a message.
    Be sure you've called `read_msg(..., nums=True)` to ensure you know the line numbers.
- &`msg_del_lines`: Delete a range of lines from a message. Be sure you've called `read_msg(..., nums=True)` to ensure you know the line numbers.
- &`load_gist`: Retrieve a gist
- &`gist_file`: Get the first file from a gist
- &`is_usable_tool`: True if the function has a docstring and all parameters have types, meaning that it can be used as an LLM tool.
- &`import_gist`: Import gist directly from string without saving to disk
- &`doimport`: Import a module into the caller's global namespace
- &`resolve`: Resolve a dotted symbol string to its Python object, with optional [n] indexing. Sets global `_last` to the resolved object for chaining. Pass `"_last"` to reference the result of the previous tool call.

    Examples:

    - `resolve("sympy.sets.sets.Interval")` -> `<class 'sympy.sets.sets.Interval'>`
    - `resolve("mylist[2]")` -> third element of mylist
- &`symsrc`: Get the source code for a symbol.

    Examples:

    - `symsrc("sympy.sets.sets.Interval")` -> source code of Interval class
    - `symsrc("_last")` -> source of object from previous tool call
    - For dispatchers or registries of callables: `getnth("module.dispatcher.funcs", n) then symsrc("_last")`
- &`showsrc`: Create a note to show the user (and following LLM prompts) the source of `sym`, following `symsrc` rules
- &`gettype`: Get the type of a symbol and set `_last`.

    Examples:

    - `gettype("sympy.sets.sets.Interval")` -> `<class 'type'>`
    - `gettype("_last")` -> type of previous result
- &`getdir`: Get dir() listing of a symbol's attributes and set `_last`. E.g: `getdir("sympy.Interval")` -> `['__add__', '__and__', ...]`
- &`getval`: Get repr of a symbol's value and set `_last`.

    Examples:
    
    - `getval("sympy.sets.sets.Interval")` -> `<class 'sympy.sets.sets.Interval'>`
    - `getval("some_dict.keys")` -> `dict_keys([...])`
- &`getnth`: Get the nth value from a dict (or any object with .values()). Sets `_last` so you can chain with `symsrc("_last")` etc.

    Examples:
    
    - `getnth("dispatcher.funcs", 12)` -> 13th registered function
    - `getnth("dispatcher.funcs", 0); symsrc("_last")` -> source of first handler
- &`run_code_interactive`: Insert code into user's dialog and request for the user to run it. Use other tools where possible, 
    but if they can not find needed information, *ALWAYS* use this instead of guessing or giving up.
    IMPORTANT: This tool is TERMINAL - after calling it, you MUST stop all tool usage 
    and wait for user response. Never call additional tools after this one.
- &`solveit_version`: Return the version of solveit if it is found
- &`in_dialog`: Check if the code is running in a solveit dialog
- &`get_caller_globals`: Return the globals of the caller

# add_to_namespace

In [ ]:
#| export
def setup_ns(ns=None, **kwargs):
    "Add `kwargs` to the namespace `ns` or current dialog"
    ns = ns or _find_frame_dict('__msg_id')
    # lc = list(locals().items())[1:]
    thisid = find_msg_id()
    for k,v in kwargs.items(): ns[k] = v
    msgid = add_msg('\n\n'.join(f"{k} = {v}" for k,v in kwargs.items()))
    print(thisid, msgid)

setup_dialog = setup_ns

In [ ]:
setup_ns(
    ORG='', 
    REPO='', 
    USERNAME='civvic', 
    DESCRIPTION='', 
    COMMIT_MESSAGE='', 
    BRANCH_NAME='',
)

_2bb1150b _4d523b62


ORG = 

REPO = 

USERNAME = civvic

DESCRIPTION = 

COMMIT_MESSAGE = 

BRANCH_NAME = 

In [ ]:
test_is('USERNAME' in globals(), True)

# Messages

In [ ]:
find_var??


```python
def find_var(var:str):
    "Search for var in all frames of the call stack"
    return _find_frame_dict(var)[var]
```

**File:** `/usr/local/lib/python3.12/site-packages/dialoghelper/core.py`

In [ ]:
print((msg := read_msg(-1, True)['msg'])['output'][0]['data']['text/plain'])

Signature: find_var(var: str)
Source:   
def find_var(var:str):
    "Search for var in all frames of the call stack"
    return _find_frame_dict(var)[var]
File:      /usr/local/lib/python3.12/site-packages/dialoghelper/core.py
Type:      function


In [ ]:
print(msg['output'][0]['data']['text/markdown'])


```python
def find_var(var:str):
    "Search for var in all frames of the call stack"
    return _find_frame_dict(var)[var]
```

**File:** `/usr/local/lib/python3.12/site-packages/dialoghelper/core.py`


### hack resolve()

Until PR [#76](https://github.com/AnswerDotAI/dialoghelper/pull/76) accepted or bug resolved.

In [ ]:
def resolve(
    sym: str  # Dotted symbol path, with optional [n] indexing, e.g. "module.attr.subattr[1]" or "_last" for previous result
):
    """Resolve a dotted symbol string to its Python object, with optional [n] indexing. Sets global `_last` to the resolved object for chaining. Pass `"_last"` to reference the result of the previous tool call.

    Examples:

    - `resolve("sympy.sets.sets.Interval")` -> `<class 'sympy.sets.sets.Interval'>`
    - `resolve("mylist[2]")` -> third element of mylist"""
    global _last
    if (sym := sym.strip()) == '_last': return _last
    g = _find_frame_dict('__msg_id')
    if match := re.match(r'^(\w+)\[(\d+)\]$', sym):
        attr, idx = match.groups()
        parts, _last = ['_last'], _last[int(idx)] if attr == '_last' else g[attr][int(idx)]
    else: parts = re.split(r'\.(?![^\[]*\])', sym)
    obj = _last if parts[0] == '_last' else g[parts[0]]
    for part in parts[1:]:
        match = re.match(r'(\w+)\[(\d+)\]$', part)
        if match:
            attr, idx = match.groups()
            obj = getattr(obj, attr)[int(idx)]
        else: obj = getattr(obj, part)
    _last = obj
    return obj

In [ ]:
myobj  = dict2obj({'abc': ['a', 'b', 'c']})
resolve('myobj.abc[2]')

'c'

In [ ]:
mylist = ['a', 'b', 'c']
resolve('mylist[2]')

'c'

Test prompt.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Beginnings, ...

```python
print("I'm excised!")
```

in between two fenced code blocks,  

```python
print("Me too!")
```

and ends.

In [ ]:
msg = read_msg()
if isinstance(o := resolve('msg.msg.output'), str): print(o)

Beginnings, ...

```python
print("I'm excised!")
```

in between two fenced code blocks,  

```python
print("Me too!")
```

and ends.


In [ ]:
msg = read_msg()
o = resolve('msg.msg.output')
print(o)

[{'output_type': 'stream', 'name': 'stdout', 'text': 'Beginnings, ...\n\n```python\nprint("I\'m excised!")\n```\n\nin between two fenced code blocks,  \n\n```python\nprint("Me too!")\n```\n\nand ends.\n'}]


In [ ]:
resolve('o[0]')

```python
{ 'name': 'stdout',
  'output_type': 'stream',
  'text': 'Beginnings, ...\n'
          '\n'
          '```python\n'
          'print("I\'m excised!")\n'
          '```\n'
          '\n'
          'in between two fenced code blocks,  \n'
          '\n'
          '```python\n'
          'print("Me too!")\n'
          '```\n'
          '\n'
          'and ends.\n'}
```

In [ ]:
%%markdown
### h3

### h3


In [ ]:
msg = read_msg()
o = dict2obj(resolve('msg.msg.output[0]'))
display(o)
resolve('o.data.text/markdown')


```python
{ 'data': { 'text/markdown': '### h3\n',
            'text/plain': '<IPython.core.display.Markdown object>'},
  'metadata': {},
  'output_type': 'display_data'}
```

'### h3\n'

prompt output is a string.

code output is list of dicts (.ipynb JSON compatible)

In [ ]:
msg = read_msg()
o = resolve('msg.msg.output')
print(o)

note output is empty.

In [ ]:
msg = read_msg()
resolve('msg.msg.output')

''

In [ ]:
def get_output(msgid:str=None) -> list[str]:
    msg = read_msg(n=0, relative=True, msgid=msgid or find_msg_id())
    if output := resolve('msg.msg.output'):
        if isinstance(output, str): return output. # prompt
        
        
    if isinstance(o, str): return [o]
    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

# summarize

In [ ]:
def summarize(target, context): pass

# export -

In [ ]:
from dutil.flakes import add_flakes

In [ ]:
add_flakes()

<!-- flakes report: _9e1c6a04 -->
<span hx-on-click="setTimeout(() => selectMsg($('#_e72f67fd'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_e72f67fd**</span>  

- **UnusedImport**: 1  
    `(6, 1)`: `fastcore.all as FC` imported but unused  
- **ImportStarUsed**: 1  
    `(9, 1)`: `from dialoghelper import *` used; unable to detect undefined names  

<span hx-on-click="setTimeout(() => selectMsg($('#_9027bb28'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_9027bb28**</span>  

- **ImportStarUsage**: 1  
    `(62, 20)`: `is_usable_tool` may be undefined, or defined from star imports: dialoghelper  

<span hx-on-click="setTimeout(() => selectMsg($('#_6a2bafc4'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_6a2bafc4**</span>  

- **ImportStarUsage**: 4  
    `(68, 5)`: `add_msg` may be undefined, or defined from star imports: dialoghelper  
    `(68, 13)`: `mk_toollist` may be undefined, or defined from star imports: dialoghelper  
    `(74, 14)`: `find_msg_id` may be undefined, or defined from star imports: dialoghelper  
    `(76, 13)`: `add_msg` may be undefined, or defined from star imports: dialoghelper  



In [ ]:
#|hide
#|eval: false
import fastcore.all as FC
from nbdev import nbdev_export
if FC.IN_NOTEBOOK:
    nb_path = '00_core.ipynb'
    nbdev_export(nb_path)